In [15]:
import pandas as pd


df_initial=pd.read_csv("one_step.csv",index_col=False)
df_best=pd.read_csv("best_gpt_prompt_generations.csv",index_col=False)
for i in range(20):
    print("products: ")
    print(df_initial["input"][i])
    print("generation_1: ")
    print(df_initial["response"][i])
    print("generation_2: ")
    print(df_best["response"][i])
    print("\n\n\n\n")

products: 
<product> Yamaha HTR-5660 6-Channel Digital Home Theater Receiver (Discontinued by Manufacturer)
<features> ['6.1-channel digital receiver with 85 watts per channel (x 6)', 'Decoding for DTS-ES Discrete 6.1, DTS-ES Matrix 6.1, DTS Neo:6, Dolby Digital EX, and Dolby Pro Logic II', '24 distinct sound programs with 44 variations meet every sound need', '4 optical and 1 coaxial digital inputs; 5 S-Video inputs and 2 outputs; 2 component-video inputs; set of front-panel A/V inputs', 'Measures 17-1/8 x 6-7/16 x 15-3/8 inches (W x H x D) and weighs 25 pounds']

<product> Sony KV-20FV12 20&quot; Trinitron Wega Flat-Screen TV
<features> ['20-inch TV with Trinitron flat picture tube; 22.25 x 18.38 x 19.88 inches (W x H x D)', '3-line digital comb filter improves image resolution and clarity', 'SpeedSurf rapid channel surfing, 5 favorite channel presets, and sleep timer', '2 sets of composite inputs (1 front and 1 rear), S-video input, audio output, and headphone jack', '10-watt DAC sp

In [1]:
import openai
import os
import credentials
os.environ["OPENAI_API_KEY"] = credentials.api_key
openai.api_key = os.getenv("OPENAI_API_KEY")
from utils.prompter import Prompter
import pandas as pd
from tqdm import tqdm

cables=pd.read_csv("../amazon-dataset/cables.csv")
amps=pd.read_csv("../amazon-dataset/home_audio.csv")
teles=pd.read_csv("../amazon-dataset/televisions.csv")
cables = cables.sample(frac=1, random_state=10).reset_index(drop=True)[:20]
amps = amps.sample(frac=1, random_state=10).reset_index(drop=True)[:20]
teles = teles.sample(frac=1, random_state=10).reset_index(drop=True)[:20]

eval_prompt="""
[Instruction]
Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of the response. Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the response on a scale of 1 to 10 by strictly following this format: \"[[rating]]\", for example: \"Rating: [[5]]\".

[Question]
I'll give you as "input" a sequence of products with their functionalities. Each product is in the form:

<product> {product name}
<features> {list of features of the product}

You have to write a description of a luxury ship room containing these products. Do not copy the features, try to focus on the user experience instead of the technical details.
Write it with an engaging tone for the ship website.


"""

prod_list=[prod for prod in amps["title"]]+[prod for prod in teles["title"]]+[prod for prod in cables["title"]]
spec_list=[feat for feat in amps["feature"]]+[feat for feat in teles["feature"]]+[feat for feat in cables["feature"]]
input=list()
assert len(prod_list)==len(spec_list)
for i in tqdm(range(20)):
    input.append("<product> "+prod_list[i]+"\n"+"<features> "+spec_list[i]+"\n\n"+"<product> "+prod_list[20+i]+"\n"+"<features> "+spec_list[20+i]+"\n\n"+"<product> "+prod_list[40+i]+"\n"+"<functionalities> "+spec_list[40+i]+"\n\n")


100%|██████████| 20/20 [00:00<00:00, 196915.68it/s]


In [14]:
import generate
from generate import generate_one_step, generate_two_step
instruction="""
I'll provide you with an 'input' consisting of a sequence of products along with their functionalities. Each product will be presented in the following format:

<product> {product name}
<features> {list of features of the product}

Your task is to craft a captivating description of a luxurious ship room that incorporates all of these products seamlessly. Ensure that you include each of these products in your description, emphasizing the user experience rather than delving into technical details. Create the description with an engaging and enticing tone suitable for the ship's website.
"""
df_one=generate_one_step(instruction)
#df_two=generate_two_step()

Generating: 100%|██████████| 20/20 [03:30<00:00, 10.53s/it]


In [15]:
responses=df_one["response"]
val_list=list()
for i in tqdm(range(20)):
    one_prompt=eval_prompt+"Input:\n"+input[i]+"[The Start of Assistant's Answer]\n"+responses[i]+"\n[The End of Assistant's Answer]"
    one_val = openai.ChatCompletion.create(
        model="gpt-4",
        temperature=0,
        messages= [ {"role": "system","content": "You are a helpful assistant."},
                    {"role": "user","content": one_prompt}])
    val_list.append(int(one_val["choices"][0]["message"]["content"].split("[[")[1].split("]]")[0]))
df_one["eval"]=val_list

100%|██████████| 20/20 [03:09<00:00,  9.47s/it]


In [16]:
df_one

,input,response,eval
0,<product> Yamaha HTR-5660 6-Channel Digital Ho...,The luxury cruise liner offers its passengers ...,7
1,<product> Cerwin-Vega CLSC-6 Classic Series 6....,The luxury cruise liner has created a stunning...,2
2,<product> Advent MARBL-1 Indoor/Outdoor Speake...,The luxury cruise liner offers guests the ulti...,6
3,<product> JBL Northridge EC35 3-Way Dual 5-Inc...,The luxury cruise liner offers its passengers ...,6
4,<product> KLH 943PL 100-Watt 3-Way Center Chan...,The luxury cruise liner offers its passengers ...,6
5,<product> KLH ASW10-120 Powered Subwoofer (Dis...,The luxury cruise liner has it all - from stat...,8
6,<product> Sony DVP-NC600 5-Disc Carousel Chang...,The ultimate luxury cruising experience can no...,6
7,<product> Panasonic SC-DK2 5-DVD Home Theater ...,The luxury cruise liner offers its guests a tr...,6
8,<product> Sangean DT-110GC Digital Pocket AM/F...,The luxury cruise liner has it all! From state...,8
9,<product> Phoenix Gold ISM-6 Single Source 6-Z...,The Phoenix Gold ISM-6 Single Source 6-Zone Sp...,6


In [17]:
np.mean(df_one["eval"])

6.55

In [19]:
df_one["response"][8]

'The luxury cruise liner has it all! From state-of-the-art technology to elegant furnishings, this cabin is sure to impress even the most discerning traveler. The Sangean DT-110GC Digital Pocket AM/FM Stereo Receiver allows guests to enjoy music and radio programs while relaxing in comfort. With its 15 presets on both FM and AM stations, as well as three random settings, there’s no need to worry about missing out on any favorite shows. For those who prefer to listen privately, the included headphones are perfect for enjoying some quiet time. The Samsung TXN2775HF 27” HD-Ready TV with DynaFlat CRT Screen ensures that guests can watch movies and television shows in crystal clear quality. Its 27” display makes viewing more immersive and comfortable, while its DynaFlat screen reduces reflection and glare so guests won’t have to squint when watching their favorite show. Guests can also connect their own devices such as game consoles, camcorders, and computers via the side panel AV hookup. F

In [ ]:
df_one=pd.read_csv("./one_step.csv")
df_two=pd.read_csv("./two_step.csv")
one_step_response=df_one["response"]
two_step_response=df_two["response"]
one_val_list=list()
two_val_list=list()
for i in tqdm(range(20)):
    one_prompt=eval_prompt+"Input:\n"+input[i]+"[The Start of Assistant's Answer]\n"+one_step_response[i]+"\n[The End of Assistant's Answer]"
    one_val = openai.ChatCompletion.create(
        model="gpt-4",
        temperature=0,
        messages= [ {"role": "system","content": "You are a helpful assistant."},
                    {"role": "user","content": one_prompt}])
    one_val_list.append(int(one_val["choices"][0]["message"]["content"].split("[[")[1].split("]]")[0]))
    two_prompt=eval_prompt+"Input:\n"+input[i]+"[The Start of Assistant's Answer]\n"+two_step_response[i]+"\n[The End of Assistant's Answer]"
    two_val = openai.ChatCompletion.create(
        model="gpt-4",
        temperature=0,
        messages= [ {"role": "system","content": "You are a helpful assistant."},
                    {"role": "user","content": two_prompt}])
    two_val_list.append(int(two_val["choices"][0]["message"]["content"].split("[[")[1].split("]]")[0]))
df_one["eval"]=one_val_list
df_one.to_csv("one_step.csv",index=False)
df_two["eval"]=two_val_list
df_two.to_csv("two_step.csv",index=False)

In [4]:
import numpy as np


print(np.mean(pd.read_csv("one_step.csv")["eval"]))
print(np.mean(pd.read_csv("two_step.csv")["eval"]))
print(np.mean(pd.read_csv("best_gpt_prompt_generations.csv")["eval"]))


6.3
5.35
6.45


In [6]:
df=pd.read_csv("two_step.csv")
df

,input_1,response,eval
0,<product> Yamaha HTR-5660 6-Channel Digital Ho...,The luxurious cabin on this cruise liner comes...,4
1,<product> Cerwin-Vega CLSC-6 Classic Series 6....,The luxurious cabin on this cruise liner is eq...,7
2,<product> Advent MARBL-1 Indoor/Outdoor Speake...,The luxurious cabin aboard this cruise liner c...,7
3,<product> JBL Northridge EC35 3-Way Dual 5-Inc...,The luxury cabin on this cruise liner offers g...,3
4,<product> KLH 943PL 100-Watt 3-Way Center Chan...,The luxurious cabin on this cruise liner conta...,7
5,<product> KLH ASW10-120 Powered Subwoofer (Dis...,The luxurious cabin on this cruise liner comes...,6
6,<product> Sony DVP-NC600 5-Disc Carousel Chang...,The luxurious cabin on this cruise ship boasts...,5
7,<product> Panasonic SC-DK2 5-DVD Home Theater ...,The luxurious cabin on this cruise liner featu...,7
8,<product> Sangean DT-110GC Digital Pocket AM/F...,The luxurious cabin aboard our cruise liner fe...,7
9,<product> Phoenix Gold ISM-6 Single Source 6-Z...,The luxurious cabin aboard our cruise liner of...,7


In [10]:
df["response"][14]

'The luxurious cabin on board our cruise liner features state-of-the-art entertainment systems. Our guests will be able to watch their favourite films and listen to their favourite tunes in stunning 5.1 channel surround sound. We also offer a selection of gaming consoles so they can play their favourite games while relaxing onboard.'